In [1]:
#!pip install sqlalchemy
#!pip install psycopg2


import re
from sqlalchemy import create_engine
import json
import pandas as pd
import numpy as np
import psycopg2
from config import db_password

In [2]:
# Define variables for database connection

protocol = "postgres"
user = "postgres"
location = "localhost"
port = "5432"
db = "movie_data"

connection_string = f'{protocol}://{user}:{db_password}@{location}:{port}/{db}'



In [4]:
# Begin the automation of the Extraction Process
# Create a function that takes in three files (wiki_movies_data, kaggle_movies_data, MovieLens_rating+data )

def etl_data(wiki_movies_raw, kaggle_metadata_raw, MoviesLens_ratings_raw):
        
        # Define a file path to where data files are located using a 
        # variable (file_dir) in this case

        file_dir = 'c:/Users/Franc/Berkeley/class_folder/Movies_ETL/Resources' 
        
        # Using with open() method, open wikipedia.movies.json file. Specify mode. 
        # This is for loading and reading the file.
        # ....................... File 1 ....................................................
        with open(f'{file_dir}/wikipedia.movies.json', mode='r') as file:
            wiki_movies_raw = json.load(file)
            len(wiki_movies_raw)
        
        # Load and Read Kaggle Metadata File using Panda Read method, which is pd.read_csv() method.
        # .........................File 2.......................................................
            kaggle_metadata = pd.read_csv(f'{file_dir}/movies_metadata.csv')
            len(kaggle_metadata)
        
        # Load and read the MovieLens rating data using Panda Read i.e pd.read() method.
        # .......................... File 3 .....................................................
            ratings = pd.read_csv(f'{file_dir}/ratings.csv')
            len(ratings)
        
        # Make a copy of the each file with a variable in dict
        wiki_data_file = dict(wiki_movies_file)
        kaggle_data_file = dict(kaggle_movies_file)
        ratings_data_file = dict(MovieLens_ratings_file)
        
        # Create a DataFrame
        wiki_movies_df = pd.DataFrame(wiki_movies_raw)
        kaggle_data_df = pd.DataFrame(kaggle_metadata_raw)
        MovieLens_ratings_df = pd.DataFrame(MovieLens_ratings_raw)
                   
        # Begin Cleaning file 1 ......wiki_movies_file  
    
        alt_titles = {}     #Create an empty list for alt_title 
                        # combine alternate titles into one list
        
        for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        
            if key in wiki_movies_file:
                alt_titles[key] = wiki_movies_file[key]
                wiki_movies_file.pop(key)
   
            if len(alt_titles) > 0:    # Use the length of alt_titles to check or validate value
                wiki_movies_file['alt_titles'] = alt_titles  # Convert to alt_titles

        # merge column names
        def change_column_name(old_name, new_name):  # Create a function change_column_name old and new 
        
            if old_name in wiki_movies_file:
                wiki_movies_file[new_name] = wiki_movies_file.pop(old_name)   # Use pop() method to consolidate columns with
                                                    # same data into one or change the name of a key
   
        change_column_name('Adaptation by', 'Writer(s)') # Go through and decide a better name for it.
        change_column_name('Country of origin', 'Country')
        change_column_name('Directed by', 'Director')
        change_column_name('Distributed by', 'Distributor')
        change_column_name('Edited by', 'Editor(s)')
        change_column_name('Length', 'Running time')
        change_column_name('Original release', 'Release date')
        change_column_name('Music by', 'Composer(s)')
        change_column_name('Produced by', 'Producer(s)')
        change_column_name('Producer', 'Producer(s)')
        change_column_name('Productioncompanies ', 'Production company(s)')
        change_column_name('Productioncompany ', 'Production company(s)')
        change_column_name('Released', 'Release Date')
        change_column_name('Release Date', 'Release date')
        change_column_name('Screen story by', 'Writer(s)')
        change_column_name('Screenplay by', 'Writer(s)')
        change_column_name('Story by', 'Writer(s)')
        change_column_name('Theme music composer', 'Composer(s)')
        change_column_name('Written by', 'Writer(s)')

        return
        
     
        
# Create a list comprehension with 
[column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
wiki_movies_df.head()
        
# Begin cleaning of Kaggle Metadata file by inspecting and checking datatype        
kaggle_metadata.dtypes

# Convert to values
kaggle_metadata['adult'].value_counts()

# Remove Bad Data from Kaggle
kaggle_metadata[~kaggle_metadata['adult'].isin(['True','False'])]




# Begin cleaning of MovieLens_ratings
      

NameError: name 'wiki_movies_df' is not defined

In [ ]:
#

In [ ]:
wiki_movies_df.head()


In [ ]:
# Parse Release Date .............................STEP 1

release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)



In [ ]:
# Extraxct the dates .............................STEP 3

release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)


In [ ]:
# Parse the dates with built_in date function. We do not need to create a function .....STEP 4

wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)




In [ ]:
# Parse running time

running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)


In [ ]:
running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE).sum()

In [ ]:
running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE) != True]

In [ ]:
running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE).sum()


In [ ]:
# Output

running_time[running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE) != True]

In [ ]:
# With our new pattern, it’s time to extract values. We only want to extract digits,
# and we want to allow for both possible patterns. Therefore, we’ll add capture groups around the \d instances as well as add an alternating character. 
# Our code will look like the following.



running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')

In [ ]:
# This new DataFrame is all strings, we’ll need to convert them to numeric values.

running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)


In [ ]:
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)

In [ ]:
# Drop the running time column as we did with all in the past



In [ ]:
# 8.3.12 Clean the Kaggle Data    First Inspect the data

kaggle_metadata.dtypes



In [ ]:
# Convert to values

kaggle_metadata['adult'].value_counts()

In [ ]:

# Remove Bad Data

kaggle_metadata[~kaggle_metadata['adult'].isin(['True','False'])]


In [ ]:
# Drop the adult column

kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')

In [ ]:
# Check to see te values of the video column

kaggle_metadata['video'].value_counts()


In [ ]:
# Convert video column

kaggle_metadata['video'] == 'True'

In [ ]:
# Assing it back to Video column after its been converted to what we want

kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'

In [ ]:
# Convert Kaggle to Numeric

kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')

In [ ]:
# Convert Release Date

kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

In [ ]:
# Checking on Ratings Data

ratings.info(null_counts=True)

In [ ]:
pd.to_datetime(ratings['timestamp'], unit='s')

In [ ]:
# 

In [ ]:
# movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])

In [ ]:
# wiki_movies_df[['title_wiki','title_kaggle']]